In [1]:
from pyspark import SparkConf, SparkContext
import sys


#Create a configuration object and
#set the name of the application
conf = SparkConf().setAppName("Spark Word Count")
# Create a Spark Context object
sc = SparkContext.getOrCreate(conf=conf)

In [2]:
folderPath = "../covid_cases_jrc.csv"
colnames = ['Date','iso3','CountryName','Region','lat','lon','CumulativePositive','CumulativeDeceased','CumulativeRecovered','CurrentlyPositive','Hospitalized','IntensiveCare','EUcountry','EUCPMcountry','NUTS']

In [3]:
logFileRDD = sc.textFile(folderPath)

In [5]:
from  mapred_utils import  CovidData
#currently positive and hospitalized not needed (the latter has too many nans)

# map to (nuts,(date, row_info) )
def map0(row):
    row_data = row.split(",")
    date = row_data[0]
    nuts = row_data[-1]
    # avoid mapping the header
    if "CumulativePositive" not in row:
        return nuts, (date, CovidData.parse(row))
    return None
def reduce0(item1, item2):
    date1, covid1 = item1
    date2, covid2 = item2
    
    if date1 >= date2:
        return date1, CovidData.time_aggregate(covid1, covid2)
    else:
        return date2, CovidData.time_aggregate(covid1, covid2)

    
def map1(item):
    nuts, (date, covid_data) = item
    #row_data = row.split(",")
    #date = row_data[0]
    #nuts = row_data[-1]
    
    if len(nuts)> 4:
        nuts = nuts[:4]
    return nuts, covid_data
    
def reduce1(covid1, covid2):
    return CovidData.aggregate(covid1, covid2)
    
    

        
    
    
    
    

nogroup_count = logFileRDD.map(map0).filter(lambda item: item is not None)\
        .reduceByKey(reduce0)\
        .map(map1).filter(lambda item: item is not None)\
        .filter(lambda item: len(item[0]) ==4 )\
        .count()

group_count = logFileRDD.map(map0).filter(lambda item: item is not None)\
        .reduceByKey(reduce0)\
        .map(map1).filter(lambda item: item is not None)\
        .filter(lambda item: len(item[0]) ==4 )\
        .groupByKey()\
        .count()


In [ ]:
print(nogroup_count)
print(group_count)

# find regions that at the latest date have no cases (missing data error, jrc side)

In [6]:
from  mapred_utils import  CovidData
res = logFileRDD.map(map0).filter(lambda item: item is not None)\
        .reduceByKey(reduce0)\
        .map(map1).filter(lambda item: item is not None)\
        .filter(lambda item: len(item[0]) ==4 )\
        .reduceByKey(reduce1)\
        .filter(lambda item: item[1].cum_positive == 0.)\
        .map(lambda item: item[1].as_row(item[0]))\
        .collect()
res

ImportError: cannot import name 'DateContainer' from 'mapred_utils' (C:\Users\chris\Desktop\covid_project\main\database_generator\ml_dataset_lib\mapred_utils.py)

# find all regions that can be used for analysis

In [7]:
from  mapred_utils import  CovidData
res = logFileRDD.map(map0).filter(lambda item: item is not None)\
        .reduceByKey(reduce0)\
        .map(map1).filter(lambda item: item is not None)\
        .filter(lambda item: len(item[0]) ==4 )\
        .reduceByKey(reduce1)\
        .filter(lambda item: item[1].cum_positive > 0.)\
        .map(lambda item: item[1].as_row(item[0]))\
        .collect()
res

['ITC4,856667.0,nan,nan,12455.0,283.0,35.0,ITA,Italy,True,True,2021-08-06',
 'ITH3,440420.0,nan,nan,13386.0,160.0,18.0,ITA,Italy,True,True,2021-08-06',
 'ITH5,397722.0,nan,nan,9567.0,285.0,22.0,ITA,Italy,True,True,2021-08-06',
 'ITH1,73918.0,nan,nan,363.0,10.0,1.0,ITA,Italy,True,True,2021-08-06',
 'ITG1,247040.0,nan,nan,12979.0,417.0,42.0,ITA,Italy,True,True,2021-08-06',
 'AT11,18285.0,331.0,17858.0,96.0,0.0,1.0,AUT,Austria,True,True,2021-08-05',
 'AT22,81456.0,2096.0,78755.0,605.0,14.0,4.0,AUT,Austria,True,True,2021-08-05',
 'AT33,64270.0,689.0,63068.0,513.0,8.0,4.0,AUT,Austria,True,True,2021-08-05',
 'AT13,142233.0,2369.0,135414.0,4450.0,38.0,18.0,AUT,Austria,True,True,2021-08-05',
 'ITF1,76679.0,2515.0,72504.0,1660.0,42.0,1.0,ITA,Italy,True,True,2021-08-06',
 'ITF4,257188.0,6672.0,247483.0,3033.0,109.0,15.0,ITA,Italy,True,True,2021-08-06',
 'CH04,116921.0,1420.0,0.0,115501.0,63.0,18.0,CHE,Switzerland,False,False,2021-08-06',
 'ITF6,71821.0,1265.0,67885.0,2671.0,81.0,5.0,ITA,Italy,Tr

In [ ]:
import urllib.request

covid_data_url = "https://raw.githubusercontent.com/ec-jrc/COVID-19/master/data-by-region/jrc-covid-19-all-days-by-regions.csv"
    
with urllib.request.urlopen(covid_data_url) as f:
    string = f.read().decode('utf-8')
    
